In [1]:
import re
import time
import json
import redis
import scrapy
import datetime
import pandas as pd
import codecs
from lxml import etree

from sqlalchemy import create_engine
from selenium import webdriver

pool = redis.ConnectionPool(host='172.17.0.2', port=6379, decode_responses=True, db=1, password='De32wsxC')
redis_con = redis.Redis(connection_pool=pool)

In [2]:
def assemble_list():
    """
    组合list
    """
    head = 'https://www.lagou.com/jobs/list_?px=new&gx=兼职&city='
    city = pd.read_csv('../crawl_lagou/tmp/city.csv')
    # 清空数据库
    redis_con.flushdb()
    for i in range(0, len(city)):
        city_name = city['city'][i]
        url = head + city_name + '#filterBox'
        redis_con.sadd('lagou_list', url)

In [61]:
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference('permissions.default.stylesheet', 2)
firefox_profile.set_preference('permissions.default.image', 2)
firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')

# driver = webdriver.Firefox(firefox_profile=firefox_profile)
driver = webdriver.Firefox()
# driver.set_page_load_timeout(5)

urls = {
    'login': 'https://passport.lagou.com/login/login.html?service=https%3a%2f%2fwww.lagou.com%2f',
    'city': 'https://www.lagou.com/jobs/allCity.html',
    'category': 'https://www.lagou.com/jobs/list_?px=new&gx=兼职&city=成都#filterBox'
}
driver.get(urls['login'])
elem=driver.find_elements_by_xpath("//input[@class='input input_white']")[0]
elem.send_keys('18349125880')
elem=driver.find_elements_by_xpath("//input[@class='input input_white']")[1]
elem.send_keys('ldj6692240')

In [49]:
driver.get(urls['city'])

cities = driver.find_elements_by_xpath("//ul[@class='city_list']/li/a")
city = []
for i in range(0, len(cities)):
    city.append(cities[i].text)

cities = pd.DataFrame(pd.Series(city), columns=['city'])
cities.to_csv('../crawl_lagou/tmp/city.csv', index=False)
assemble_list()
result = pd.DataFrame([],columns=['work_name','city','zone','work_href','publish_time','money','require','company_name','company_href',
                                 'industry','work_desc_simple','work_advantage'])

In [56]:
while True:
    url = redis_con.spop('lagou_list')
    if url == None:
        break
    driver.get(url)
    time.sleep(2)
    is_null = driver.find_elements_by_xpath("//span[@class='span totalNum']")
    if int(is_null[0].text) == 0:
        continue

    while True:
        elem=driver.find_elements_by_xpath("//li[@class='con_list_item first_row default_list']")
        elem.extend(driver.find_elements_by_xpath("//li[@class='con_list_item default_list']"))
        temp = pd.DataFrame([],columns=['work_name','city','zone','work_href','publish_time','money','require','company_name','company_href',
                                     'industry','work_desc_simple','work_advantage'])
        for i in range(0,len(elem)):
            temp.loc[i,'work_name'] = elem[i].find_elements_by_xpath(".//a[@class='position_link']/h3")[0].text
            temp['city'][i] = re.findall('city=(.*)#',url)[0]
            temp['zone'][i] = elem[i].find_elements_by_xpath(".//a[@class='position_link']/span/em")[0].text
            temp['work_href'][i] = elem[i].find_elements_by_xpath(".//a[@class='position_link']")[0].get_attribute('href')
            temp['publish_time'][i] = elem[i].find_elements_by_xpath(".//span[@class='format-time']")[0].text
            temp['money'][i] = elem[i].find_elements_by_xpath(".//span[@class='money']")[0].text
            temp['require'][i] = elem[i].find_elements_by_xpath(".//div[@class='p_bot']/div")[0].text
            temp['company_name'][i] = elem[i].find_elements_by_xpath(".//div[@class='company_name']/a")[0].text
            temp['company_href'][i] = elem[i].find_elements_by_xpath(".//div[@class='company_name']/a")[0].get_attribute('href')
            temp['industry'][i] = elem[i].find_elements_by_xpath(".//div[@class='industry']")[0].text
            work_desc_simple = elem[i].find_elements_by_xpath(".//div[@class='list_item_bot']/div[1]/span")
            work_desc_simple = [desc.text for desc in work_desc_simple]
            temp['work_desc_simple'][i] = work_desc_simple
            temp['work_advantage'][i] = elem[i].find_elements_by_xpath(".//div[@class='list_item_bot']/div[2]")[0].text
        result = result.append(temp,sort=False).reset_index(drop=True)
        next_page=driver.find_elements_by_xpath("//span[@class='pager_next ']")
        if len(next_page) > 0:
            next_page[0].click()
            time.sleep(2)
        else:
            break
#     print(work_name,zone,href,publish_time,money,reqire,company_name,company_href,industry,work_desc_simple,work_advantage)

In [57]:
driver.close()

In [59]:
result.to_csv('../crawl_lagou/tmp/兼职.csv',index=False)